# No Excuses Workout Optimizer (NEWO)
### Official Jupyter Notebook 

#### Web App: https://newoflask.herokuapp.com/

#### NEWO finds a 3 day workout split regimen for you that targets all of the most important muscles in the body with the fewest number of exercises given the equipment you have available.

#### Data Source: https://github.com/wger-project/wger/tree/master/wger/exercises/fixtures
#### Thank you wger.de!

In [130]:
import pandas as pd
import random

In [131]:
muscles = pd.read_json("muscles.json")
equipment = pd.read_json("equipment.json")
categories = pd.read_json("categories.json")
exercises = pd.read_json("exercises.json")

In [132]:
## Function that finds if there is any common data between two lists (intersection = not empty set)
def common_data(list1, list2): 
    result = False
    for x in list1: 
        for y in list2:     
            if x == y: 
                result = True
                return result            
    return result

In [133]:
#### Clean up categories dataframe
for i in categories.index:
    categories.at[i, "fields"] = categories.fields[i]["name"]
categories.drop(['model'], axis=1, inplace=True)
categories.rename(columns={'fields': 'category'}, inplace=True)

In [134]:
categories

,category,pk
0,Arms,8
1,Legs,9
2,Abs,10
3,Chest,11
4,Back,12
5,Shoulders,13
6,Calves,14


In [135]:
#### Clean up muscles dataframe
for i in muscles.index:
    name = muscles.fields[i]["name"]
    is_front = muscles.fields[i]["is_front"]
    muscles.at[i, "fields"] = name
    muscles.at[i, "is_front"] = is_front

muscles.drop(['model'], axis=1, inplace=True)
muscles.rename(columns={'fields': 'muscle'}, inplace=True)

In [136]:
muscles

,muscle,pk,is_front
0,Anterior deltoid,2,True
1,Biceps brachii,1,True
2,Biceps femoris,11,False
3,Brachialis,13,True
4,Erector spinae,16,False
5,Gastrocnemius,7,False
6,Gluteus maximus,8,False
7,Latissimus dorsi,12,False
8,Obliquus externus abdominis,14,True
9,Pectoralis major,4,True


In [137]:
#### Clean up equipment dataframe
for i in equipment.index:
    equipment.at[i, "fields"] = equipment.fields[i]["name"]

equipment.drop(['model'], axis=1, inplace=True)
equipment.rename(columns={'fields': 'category'}, inplace=True)
equipment = equipment.append({'category' : 'Squat Rack', 'pk' : 11}, ignore_index=True)

In [138]:
equipment

,category,pk
0,Barbell,1
1,SZ-Bar,2
2,Dumbbell,3
3,Gym mat,4
4,Swiss Ball,5
5,Pull-up bar,6
6,none (bodyweight exercise),7
7,Bench,8
8,Incline bench,9
9,Kettlebell,10


In [139]:
#### Clean up exercises data frame

indices_to_delete = []

for i in exercises.index:
    try:
        if(exercises.fields[i]["language"] != 2):
            indices_to_delete.append(i)
    except:
        indices_to_delete.append(i)

exercises = exercises.drop(exercises.index[indices_to_delete])
exercises["name"] = pd.Series(str,index=exercises.index)
exercises["muscles"] = pd.Series(list,index=exercises.index)
exercises["muscles_secondary"] = pd.Series(list,index=exercises.index)
exercises["equipment"] = pd.Series(list,index=exercises.index)
exercises["category"] = pd.Series(int,index=exercises.index)

for i in exercises.index:
    exercises.at[i, "name"] = exercises.fields[i]["name"]
    exercises.at[i, "muscles"] = exercises.fields[i]["muscles"]
    exercises.at[i, "muscles_secondary"] = exercises.fields[i]["muscles_secondary"]
    exercises.at[i, "equipment"] = exercises.fields[i]["equipment"]
    exercises.at[i, "category"] = exercises.fields[i]["category"]
    
exercises.drop(['fields', 'model', 'pk'], axis=1, inplace=True)
exercises.reset_index(drop=True, inplace=True)

count = 0
i_list = []
for i in exercises.index:
    if ((len(exercises.at[i, "equipment"]) == 0) or ((len(exercises.at[i, "muscles"]) == 0)) and (len(exercises.at[i, "muscles_secondary"]) == 0)) or (exercises.at[i, "equipment"] == [7]) or (4 in exercises.at[i, "equipment"]) or (exercises.at[i, "muscles"] == exercises.at[i, "muscles_secondary"]) or (len(exercises.at[i, "muscles"]) == 0) or (exercises.at[i, "category"] == 14):
        i_list.append(i)
        
exercises = exercises.drop(i_list)
exercises.reset_index(drop=True, inplace=True)

## Light explicit cleanup of exercises that need edits
exercises.at[48, "name"] = "Farmer Walks"
exercises.at[49, "name"] = "Duck Walks"
exercises.at[49, "muscles"] = [8,10]
exercises.at[49, "muscles_secondary"] = [11]

## Yolk Walks
exercises.at[50, "name"] = "Yolk Walks"

## Axe Hold
exercises.at[51, "name"] = "Axe Hold"

## Squats
exercises.at[15, "equipment"] = [1, 11]
exercises.at[15, "name"] = "Back Squat"
exercises.at[15, "muscles"] = [10, 8, 16]
exercises.at[15, "muscles_secondary"] = [12]

## Remove Super Squat
exercises = exercises.drop([37])

In [140]:
def new_exercise(n, m, ms, e, c):
    return {"name": n, "muscles": m, "muscles_secondary": ms, "equipment": e, "category": c}

In [141]:
### Adding calf exercises explicitly
exercises = exercises.append(new_exercise("Seated calf raise with dumbell", [15], [7], [3], 14), ignore_index=True)
exercises = exercises.append(new_exercise("Seated calf raise with barbell", [15], [7], [1], 14), ignore_index=True)
exercises = exercises.append(new_exercise("Standing Barbell Calf Raise", [15, 7], [], [1], 14), ignore_index=True)
exercises = exercises.append(new_exercise("Farmers walks on toes", [15, 7], [1,11,13,9], [3], 14), ignore_index=True)

In [142]:
### Adding Zercher Exercises (Personal favorite)
exercises = exercises.append(new_exercise("Zercher Squat", [10, 8, 9], [1, 6], [1], 9), ignore_index=True)
### Adding Front Squats
exercises = exercises.append(new_exercise("Front Squat", [10, 9], [1, 6], [1, 11], 9), ignore_index=True)

In [143]:
exercises

,name,muscles,muscles_secondary,equipment,category
0,Biceps curls with barbell,[1],[13],[1],8
1,Biceps curls with SZ-bar,[1],[13],[2],8
2,Biceps curls with dumbbell,[1],[13],[3],8
3,Dips between two benches,[5],[],[8],8
4,French press (skullcrusher) SZ-bar,[5],[],"[8, 2]",8
5,French press (skullcrusher) dumbbells,[5],[],"[8, 3]",8
6,Hammercurls,[1],[],[3],8
7,Bench press narrow grip,[5],"[2, 4]","[1, 8]",8
8,Benchpress dumbbells,[4],"[2, 5]","[8, 3]",11
9,Decline bench press barbell,[4],[],[1],11


In [144]:
### Finding how many exercises fall under each equipment

dic = {}
for i in range(1,11):
    dic[i] = 0

for row in exercises["equipment"]:
    for i in range(1,11):
        if i in row:
            dic[i] += 1
print(dic)

{1: 27, 2: 4, 3: 33, 4: 0, 5: 0, 6: 3, 7: 0, 8: 10, 9: 3, 10: 0}


In [145]:
### Replaces numbers in dataframe with corresponding values in other dataframes
## Must use with an exercise formatted dataframe
def replace_nums_with_names(c_df):
    new_df = pd.DataFrame(columns=["name", "muscles", "muscles_secondary", "equipment", "category"], index=c_df.index)
    
    for i in c_df.index:
        ## Convert 'muscles'
        m_list = c_df.at[i, "muscles"]
        
        new_muscles_list = []
        
        if type(m_list) == str:
            m_list = [int(j) for j in (m_list.strip('][').split(', '))]
        
        for num in m_list:
            new_muscles_list.append(str((muscles[muscles["pk"] == num].muscle.values)[0]))
        
        
        ## Convert 'muscles_secondary'
        m2_list = c_df.at[i, "muscles_secondary"]
        
        new_muscles2_list = []
        
        if type(m2_list) == str:
            if m2_list == "[]":
                m2_list = []
            else:
                m2_list = [int(j) for j in (m2_list.strip('][').split(', '))]
        
        for num in m2_list:
            new_muscles2_list.append(str((muscles[muscles["pk"] == num].muscle.values)[0]))
        
        
        ## Convert 'equipment'
        equip_list = c_df.at[i, "equipment"]
        
        new_equip_list = []
        
        if type(equip_list) == str:
            equip_list = [int(j) for j in (equip_list.strip('][').split(', '))]
        
        for num in equip_list:
            new_equip_list.append(str((equipment[equipment["pk"] == num].category.values)[0]))
        
        
        ## Convert 'category'
        categ = c_df.at[i, "category"]
        new_category = str((categories[categories["pk"] == categ].category.values)[0])
        
        
        
        new_df.at[i, "name"] = c_df.at[i, "name"]
        new_df.at[i, "muscles"] = new_muscles_list
        new_df.at[i, "muscles_secondary"] = new_muscles2_list
        new_df.at[i, "equipment"] = new_equip_list
        new_df.at[i, "category"] = new_category        
    return new_df

In [146]:
#### Main function
## equip [list] : list of equipment (by pk #) that user has
def main(equip):
    filtered_index = []
    for i in exercises.index:
        ## Finds exercises that are viable with equipment
        if(all(x in equip for x in exercises.at[i, "equipment"])):
            filtered_index.append(i)

    filter_equipment = pd.DataFrame(index=range(len(filtered_index)), columns=exercises.columns)
    #print(filter_equipment)
    j = 0
    for i in filtered_index:
        filter_equipment.iloc[j] = exercises.iloc[i]
        j += 1

    categorical_dict = {}
    for i in categories.index:
        df = filter_equipment[filter_equipment["category"] == categories.at[i, "pk"]]
        df = df.reindex(df.muscles.str.len().sort_values(ascending=False).index)
        df = df.reset_index(drop=True)
        categorical_dict[int(categories.at[i, "pk"])] = df


    all_muscles = list(range(1,17))
    muscles_hit = []

    #### Get primary muscle exercises to target maximal muscles 
    out_df = pd.DataFrame(columns = exercises.columns)

    ### Search primary muscles in each category
    for df in categorical_dict.values():
        for i in df.index:
            musc = df.at[i, "muscles"]
            if(not(common_data(muscles_hit, musc))):
                for mus in musc:
                    muscles_hit.append(mus)

                out_df = out_df.append(df.iloc[i])
                break

                
    #### Get secondary muscle exercises to target muscles that weren't hit
    secondary_index_add = []

    difference = list(set(all_muscles).difference(set(muscles_hit)))

    for df in categorical_dict.values():
        for i in df.index:
            if(common_data(df.at[i, "muscles_secondary"], difference)):
                common = list(set(df.at[i, "muscles_secondary"]) & set(difference))
                out_df = out_df.append(df.iloc[i])
                for element in common:
                    muscles_hit.append(element)

                difference = list(set(all_muscles).difference(set(muscles_hit)))
                
    
    ### Fill categories
    out_df = out_df.astype(str).drop_duplicates(subset=['muscles'], keep='first').reset_index()

    # Categories of out_df
    all_categ = [int(i) for i in list(out_df["category"])]    

    max_occuring_categ_occurences = all_categ.count(max(set(all_categ), key=all_categ.count))
    
    for categ in all_categ:
        if all_categ.count(categ) != max_occuring_categ_occurences:
            ### Add random row not already in out_df from category
            diff = max_occuring_categ_occurences - all_categ.count(categ)
            ## Finds random row
            random_row = categorical_dict[categ].iloc[random.randint(0, len(categorical_dict[categ])-1)]
            row_in_out_df = (out_df['name'] == random_row["name"]).any()
            for i in range(diff):
                while not row_in_out_df:
                    random_row = categorical_dict[categ].iloc[random.randint(0, len(categorical_dict[categ])-1)]
                    row_in_out_df = (out_df['name'] == random_row["name"]).any()

                    if(not row_in_out_df):
                        out_df = out_df.append(random_row, ignore_index=True)
                        break
    

    out_df["category"] = pd.to_numeric(out_df["category"])
    return replace_nums_with_names(out_df.sort_values(by=['category'], ascending=True)).reset_index(drop=True)

In [174]:
### Running main function given user has dumbbell, barbell, pull-up bar, and bench
## Will throw error if no exercises available
main([10,1,11])

,name,muscles,muscles_secondary,equipment,category
0,Barbell Triceps Extension,[Triceps brachii],"[Anterior deltoid, Pectoralis major]",[Barbell],Arms
1,Biceps curls with barbell,[Biceps brachii],[Brachialis],[Barbell],Arms
2,Zercher Squat,"[Quadriceps femoris, Gluteus maximus, Trapezius]","[Biceps brachii, Rectus abdominis]",[Barbell],Legs
3,Front Squat,"[Quadriceps femoris, Trapezius]","[Biceps brachii, Rectus abdominis]","[Barbell, Squat Rack]",Legs
4,Barbell Ab Rollout,"[Obliquus externus abdominis, Rectus abdominis]","[Anterior deltoid, Latissimus dorsi]",[Barbell],Abs
5,Decline bench press barbell,[Pectoralis major],[],[Barbell],Chest
6,Bent over rowing reverse,[Latissimus dorsi],"[Anterior deltoid, Biceps brachii]",[Barbell],Back
7,Snach,[Anterior deltoid],[],[Barbell],Shoulders
8,Standing Barbell Calf Raise,"[Soleus, Gastrocnemius]",[],[Barbell],Calves
